In [ ]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) =boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [13]:
train_data.shape

(404, 13)

In [14]:
test_data.shape

(102, 13)

In [15]:
train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [ ]:
# The network might be able to automatically adapt to such heterogeneous
# data, but it would definitely make learning more difficult. A widespread best practice
# to deal with such data is to do feature-wise normalization: for each feature in the input
# data (a column in the input data matrix), you subtract the mean of the feature and
# divide by the standard deviation, so that the feature is centered around 0 and has a
# unit standard deviation. This is easily done in Numpy

In [16]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

#test data ka kch bhi expose nhi krna that's why we used same mean
test_data -= mean
test_data /= std

In [17]:
from keras import models
from keras import layers
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',
  input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

#input_shape=(train_data.shape[1] ->13 neurons  0 likhte to row uthaleta
#akhri layer 1 ki coz egresssion

In [18]:
#k-fold validation
import numpy as np
k=4
#we made 4 partitions
num_val_samples = len(train_data) // k   #402//4->100 k 4 hisse
num_epochs = 100
all_scores = []

#ye code uthalena h,esha bs k apni mrzi se rkhlwna
#0-99 ,100 ka chunk bnya pehle wo validation baqi partial_train_data mn traing krwadi 
#agli iteration mn agle 100 ko uthaya for validation baqi training
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
  [train_data[:i * num_val_samples],
  train_data[(i + 1) * num_val_samples:]],
  axis=0)
  partial_train_targets = np.concatenate(
  [train_targets[:i * num_val_samples],
  train_targets[(i + 1) * num_val_samples:]],
  axis=0)



  model = build_model()
  model.fit(partial_train_data, partial_train_targets,
  epochs=num_epochs, batch_size=1, verbose=0)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
  all_scores.append(val_mae)

processing fold # 0
4/4 [==============================] - 0s 3ms/step - loss: 7.8222 - mae: 1.9826
processing fold # 1
4/4 [==============================] - 0s 3ms/step - loss: 12.3024 - mae: 2.5105
processing fold # 2
4/4 [==============================] - 0s 3ms/step - loss: 14.2544 - mae: 2.6394
processing fold # 3
4/4 [==============================] - 0s 2ms/step - loss: 13.8735 - mae: 2.6304


In [19]:
 all_scores

[1.9825575351715088, 2.510498285293579, 2.639432430267334, 2.6304450035095215]

In [20]:
 np.mean(all_scores) #yanin abs error 2.5 mtlb agr 1000 ka ghr tha to 2.5% dur price btai

2.440733313560486